In [11]:
import string
import random
from datetime import datetime
from flask import Flask, request, jsonify, redirect
import nest_asyncio
import threading

# Allow Flask to run inside Jupyter
nest_asyncio.apply()

app = Flask(__name__)

# In-memory database
urls = {}
id_counter = 1

# Helper: generate unique short code
def generate_short_code(length=6):
    characters = string.ascii_letters + string.digits
    while True:
        short_code = ''.join(random.choice(characters) for _ in range(length))
        if short_code not in urls:
            return short_code

# 1️⃣ Create short URL
@app.route('/shorten', methods=['POST'])
def create_short_url():
    global id_counter
    data = request.get_json()
    if not data or 'url' not in data:
        return jsonify({"error": "URL is required"}), 400
    short_code = generate_short_code()
    now = datetime.utcnow().isoformat()
    urls[short_code] = {
        "id": str(id_counter),
        "url": data["url"],
        "shortCode": short_code,
        "createdAt": now,
        "updatedAt": now,
        "accessCount": 0
    }
    id_counter += 1
    return jsonify(urls[short_code]), 201

# 2️⃣ Retrieve original URL via API
@app.route('/shorten/<short_code>', methods=['GET'])
def get_original_url(short_code):
    if short_code not in urls:
        return jsonify({"error": "Short URL not found"}), 404
    urls[short_code]["accessCount"] += 1
    return jsonify(urls[short_code]), 200

# 3️⃣ Update short URL
@app.route('/shorten/<short_code>', methods=['PUT'])
def update_short_url(short_code):
    if short_code not in urls:
        return jsonify({"error": "Short URL not found"}), 404
    data = request.get_json()
    if not data or "url" not in data:
        return jsonify({"error": "URL is required"}), 400
    urls[short_code]["url"] = data["url"]
    urls[short_code]["updatedAt"] = datetime.utcnow().isoformat()
    return jsonify(urls[short_code]), 200

# 4️⃣ Delete short URL
@app.route('/shorten/<short_code>', methods=['DELETE'])
def delete_short_url(short_code):
    if short_code not in urls:
        return jsonify({"error": "Short URL not found"}), 404
    del urls[short_code]
    return '', 204

# 5️⃣ Get URL statistics
@app.route('/shorten/<short_code>/stats', methods=['GET'])
def get_stats(short_code):
    if short_code not in urls:
        return jsonify({"error": "Short URL not found"}), 404
    return jsonify(urls[short_code]), 200

# 6️⃣ Browser redirect
@app.route('/r/<short_code>', methods=['GET'])
def redirect_short_url(short_code):
    if short_code not in urls:
        return "Short URL not found", 404
    urls[short_code]["accessCount"] += 1
    return redirect(urls[short_code]["url"])

# Run Flask in background thread
def run_app():
    app.run(port=5000, debug=False, use_reloader=False)

thread = threading.Thread(target=run_app)
thread.start()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


In [12]:
import requests
BASE_URL = "http://127.0.0.1:5000"

resp = requests.post(f"{BASE_URL}/shorten", json={"url": "https://www.example.com"})
data = resp.json()
short_code = data["shortCode"]
print("Short Code:", short_code)
print("Full API retrieve URL:", f"{BASE_URL}/shorten/{short_code}")
print("Redirect URL:", f"{BASE_URL}/r/{short_code}")


Short Code: 0vf3Kv
Full API retrieve URL: http://127.0.0.1:5000/shorten/0vf3Kv
Redirect URL: http://127.0.0.1:5000/r/0vf3Kv


In [13]:
resp = requests.get(f"{BASE_URL}/shorten/{short_code}")
print(resp.json())


{'accessCount': 2, 'createdAt': '2025-09-04T23:05:54.742783', 'id': '1', 'shortCode': '0vf3Kv', 'updatedAt': '2025-09-04T23:05:54.742783', 'url': 'https://www.example.com'}


In [14]:
resp = requests.get(f"{BASE_URL}/shorten/{short_code}/stats")
print(resp.json())


{'accessCount': 3, 'createdAt': '2025-09-04T23:05:54.742783', 'id': '1', 'shortCode': '0vf3Kv', 'updatedAt': '2025-09-04T23:05:54.742783', 'url': 'https://www.example.com'}
